# AI RAG Agent with FAISS and LangGraph

**Goal:** Fully autonomous AI agent system that uses LLMs and vector search to assign drivers optimally.

## Architecture

This notebook implements a **fully agentic** multi-agent system where:
- Each agent uses **LLMs** for reasoning and decision-making
- **FAISS vector database** stores driver and load embeddings
- Agents can **query** the vector store semantically
- Agents have **tools** to retrieve and analyze data
- **Autonomous** decision-making with explanations

## Problem Statement

Traditional manual driver assignment leads to:
- Manual effort and delays
- Outdated HOS data
- Driver-load mismatches
- Compliance risks

## Solution: Agentic AI with Vector Search

## 1. Setup and Imports

In [ ]:
# Install required packages
# !pip install langgraph langchain langchain-openai langchain-community faiss-cpu openai tiktoken pandas matplotlib numpy

In [ ]:
import os
from typing import TypedDict, List, Dict, Optional, Annotated, Literal
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import json
from datetime import datetime
import pandas as pd
from IPython.display import Image, display, Markdown
import matplotlib.pyplot as plt
import numpy as np
import operator

print("✓ Imports successful")

## 2. Configure LLM and Embeddings

Set your OpenAI API key to enable the agentic system.

In [ ]:
# Set your OpenAI API key
# Option 1: Set environment variable
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Option 2: Load from .env file
# from dotenv import load_dotenv
# load_dotenv()

# Check if API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not set. Please set it to run the agentic system.")
    print("   You can set it with: os.environ['OPENAI_API_KEY'] = 'your-key'")
else:
    print("✓ OpenAI API key configured")

# Initialize LLM and embeddings
llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✓ LLM and embeddings initialized")

## 3. Sample Dataset

In [ ]:
# Sample dataset from the use case
DATASET = {
    "drivers": [
        {"driver_id": "D001", "name": "Arjun Kumar", "city": "Coimbatore", "hos_remaining": 6, "skill": "Reefer", "distance_km": 12, "on_time": 92, "status": "AVAILABLE"},
        {"driver_id": "D002", "name": "Ramesh P", "city": "Erode", "hos_remaining": 4, "skill": "Flatbed", "distance_km": 50, "on_time": 88, "status": "AVAILABLE"},
        {"driver_id": "D003", "name": "Manoj S", "city": "Salem", "hos_remaining": 2, "skill": "DryVan", "distance_km": 25, "on_time": 79, "status": "BREAK"},
        {"driver_id": "D004", "name": "Karthik R", "city": "Coimbatore", "hos_remaining": 8, "skill": "DryVan", "distance_km": 10, "on_time": 96, "status": "AVAILABLE"},
        {"driver_id": "D005", "name": "Siva R", "city": "Tiruppur", "hos_remaining": 7, "skill": "Reefer", "distance_km": 34, "on_time": 90, "status": "AVAILABLE"},
        {"driver_id": "D006", "name": "Prem K", "city": "Hosur", "hos_remaining": 3, "skill": "Hazmat", "distance_km": 120, "on_time": 87, "status": "AVAILABLE"},
        {"driver_id": "D007", "name": "Vignesh", "city": "Cochin", "hos_remaining": 5, "skill": "DryVan", "distance_km": 160, "on_time": 95, "status": "AVAILABLE"},
        {"driver_id": "D008", "name": "Rahul", "city": "Pollachi", "hos_remaining": 6, "skill": "Flatbed", "distance_km": 22, "on_time": 82, "status": "AVAILABLE"}
    ],
    "loads": [
        {"load_id": "L1001", "pickup_city": "Coimbatore", "pickup_time": "2025-11-21 09:00", "weight_kg": 1200, "required_skill": "DryVan", "delivery_city": "Bangalore", "distance_km": 360},
        {"load_id": "L1002", "pickup_city": "Tiruppur", "pickup_time": "2025-11-21 13:00", "weight_kg": 800, "required_skill": "Reefer", "delivery_city": "Chennai", "distance_km": 430}
    ]
}

print("✓ Dataset loaded")
display(Markdown("### Drivers"))
display(pd.DataFrame(DATASET["drivers"]))
display(Markdown("### Loads"))
display(pd.DataFrame(DATASET["loads"]))

## 4. Build FAISS Vector Database

Create embeddings and store drivers in FAISS for semantic search.

In [ ]:
def create_driver_text_representation(driver: Dict) -> str:
    """Create rich text representation of driver for embedding."""
    return f"""
    Driver: {driver['name']} (ID: {driver['driver_id']})
    Location: {driver['city']}
    Status: {driver['status']}
    Skills: {driver['skill']} trailer operations
    Hours of Service Remaining: {driver['hos_remaining']} hours
    Distance to pickup: {driver['distance_km']} km
    On-time performance: {driver['on_time']}%
    Reliability rating: {'Excellent' if driver['on_time'] >= 90 else 'Good' if driver['on_time'] >= 80 else 'Average'}
    """.strip()

def create_load_text_representation(load: Dict) -> str:
    """Create rich text representation of load for embedding."""
    return f"""
    Load: {load['load_id']}
    Pickup: {load['pickup_city']} at {load['pickup_time']}
    Delivery: {load['delivery_city']}
    Required trailer type: {load['required_skill']}
    Weight: {load['weight_kg']} kg
    Distance: {load['distance_km']} km
    """.strip()

# Create text representations
driver_texts = [create_driver_text_representation(d) for d in DATASET["drivers"]]
driver_metadatas = [{**d} for d in DATASET["drivers"]]

# Build FAISS vector store
print("Building FAISS vector database...")
driver_vectorstore = FAISS.from_texts(
    texts=driver_texts,
    embedding=embeddings,
    metadatas=driver_metadatas
)

print(f"✓ FAISS vector database created with {len(driver_texts)} drivers")
print(f"  Vector dimension: {embeddings.embed_query('test').shape if hasattr(embeddings.embed_query('test'), 'shape') else 'N/A'}")

## 5. Define Tools for Agents

Create tools that agents can use to query the vector database and retrieve information.

In [ ]:
@tool
def search_available_drivers(query: str, k: int = 5) -> str:
    """
    Search for available drivers using semantic search.
    
    Args:
        query: Natural language query describing driver requirements
        k: Number of drivers to return (default: 5)
    
    Returns:
        JSON string with matching drivers
    """
    results = driver_vectorstore.similarity_search_with_score(query, k=k)
    
    drivers = []
    for doc, score in results:
        metadata = doc.metadata
        # Only return AVAILABLE drivers
        if metadata.get('status') == 'AVAILABLE':
            drivers.append({
                **metadata,
                'similarity_score': float(score),
                'description': doc.page_content
            })
    
    return json.dumps(drivers, indent=2)

@tool
def get_load_details(load_id: str) -> str:
    """
    Retrieve detailed information about a specific load.
    
    Args:
        load_id: The load identifier (e.g., 'L1001')
    
    Returns:
        JSON string with load details
    """
    load = next((l for l in DATASET["loads"] if l["load_id"] == load_id), None)
    
    if not load:
        return json.dumps({"error": f"Load {load_id} not found"})
    
    return json.dumps(load, indent=2)

@tool
def get_driver_by_id(driver_id: str) -> str:
    """
    Retrieve detailed information about a specific driver.
    
    Args:
        driver_id: The driver identifier (e.g., 'D001')
    
    Returns:
        JSON string with driver details
    """
    driver = next((d for d in DATASET["drivers"] if d["driver_id"] == driver_id), None)
    
    if not driver:
        return json.dumps({"error": f"Driver {driver_id} not found"})
    
    return json.dumps(driver, indent=2)

@tool
def calculate_compatibility_score(driver_id: str, load_id: str) -> str:
    """
    Calculate detailed compatibility score between a driver and load.
    
    Args:
        driver_id: The driver identifier
        load_id: The load identifier
    
    Returns:
        JSON string with score breakdown
    """
    driver = next((d for d in DATASET["drivers"] if d["driver_id"] == driver_id), None)
    load = next((l for l in DATASET["loads"] if l["load_id"] == load_id), None)
    
    if not driver or not load:
        return json.dumps({"error": "Driver or load not found"})
    
    score_breakdown = {
        "driver_id": driver_id,
        "driver_name": driver['name'],
        "load_id": load_id,
        "distance_bonus": 40 if driver['distance_km'] <= 20 else 0,
        "hos_bonus": 30 if driver['hos_remaining'] >= 6 else 0,
        "skill_match_bonus": 20 if driver['skill'] == load['required_skill'] else 0,
        "reliability_score": round((driver['on_time'] / 10) * 10, 1),
    }
    
    score_breakdown['total_score'] = sum([
        score_breakdown['distance_bonus'],
        score_breakdown['hos_bonus'],
        score_breakdown['skill_match_bonus'],
        score_breakdown['reliability_score']
    ])
    
    # Add contextual information
    score_breakdown['analysis'] = {
        'distance_km': driver['distance_km'],
        'hos_remaining': driver['hos_remaining'],
        'skill_match': driver['skill'] == load['required_skill'],
        'on_time_percentage': driver['on_time'],
        'status': driver['status']
    }
    
    return json.dumps(score_breakdown, indent=2)

# Create tools list
tools = [
    search_available_drivers,
    get_load_details,
    get_driver_by_id,
    calculate_compatibility_score
]

print("✓ Tools created:")
for tool_obj in tools:
    print(f"  - {tool_obj.name}")

## 6. Define State Schema

In [ ]:
class AgenticDriverAssignmentState(TypedDict):
    """State for the agentic driver assignment workflow."""
    
    # Input
    load_id: str
    
    # Messages for agent communication
    messages: Annotated[List, operator.add]
    
    # Agent outputs
    load_details: Optional[Dict]
    candidate_drivers: Optional[List[Dict]]
    scored_drivers: Optional[List[Dict]]
    final_assignment: Optional[Dict]
    reasoning: Annotated[List[str], operator.add]
    
    # Control flow
    next_agent: Optional[str]

print("✓ State schema defined")

## 7. Create Agentic Nodes

Each node is a fully autonomous agent with LLM reasoning and tool access.

In [ ]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

def create_agent_node(agent_name: str, system_prompt: str):
    """Factory function to create agent nodes with specific roles."""
    
    def agent_node(state: AgenticDriverAssignmentState) -> AgenticDriverAssignmentState:
        print(f"\n{'='*80}")
        print(f"[{agent_name}] Starting...")
        print(f"{'='*80}")
        
        messages = state.get("messages", [])
        
        # Create system message
        system_msg = SystemMessage(content=system_prompt)
        
        # Invoke LLM with tools
        response = llm_with_tools.invoke([system_msg] + messages)
        
        print(f"\n[{agent_name}] Response:")
        print(response.content)
        
        # Check if agent wants to use tools
        if response.tool_calls:
            print(f"\n[{agent_name}] Using tools:")
            for tool_call in response.tool_calls:
                print(f"  - {tool_call['name']}({tool_call['args']})")
        
        return {
            **state,
            "messages": [response],
            "reasoning": [f"{agent_name}: {response.content}"]
        }
    
    return agent_node

In [ ]:
# Create specialized agent nodes

load_analyzer_agent = create_agent_node(
    agent_name="Load Analyzer",
    system_prompt="""
    You are the Load Analyzer agent. Your role is to:
    1. Retrieve detailed information about the load using available tools
    2. Analyze the requirements (pickup location, time, weight, trailer type, distance)
    3. Identify key constraints and requirements for driver selection
    4. Summarize your findings clearly
    
    Use the get_load_details tool to retrieve load information.
    Be thorough and analytical in your assessment.
    """
)

driver_search_agent = create_agent_node(
    agent_name="Driver Search Specialist",
    system_prompt="""
    You are the Driver Search Specialist. Your role is to:
    1. Use semantic search to find suitable drivers based on load requirements
    2. Consider: skill match, location proximity, HOS availability, reliability
    3. Retrieve top candidate drivers using the search_available_drivers tool
    4. Provide reasoning for why these drivers are good candidates
    
    Craft intelligent search queries that capture the essence of what makes a driver suitable.
    Return at least 3-5 candidate drivers.
    """
)

matching_specialist_agent = create_agent_node(
    agent_name="Matching Specialist",
    system_prompt="""
    You are the Matching Specialist. Your role is to:
    1. Calculate detailed compatibility scores for each candidate driver
    2. Use the calculate_compatibility_score tool for each driver
    3. Compare and analyze the scores
    4. Rank drivers based on total score and provide reasoning
    
    Consider:
    - Distance to pickup (≤20km gets bonus)
    - Hours of Service remaining (≥6 hrs gets bonus)
    - Skill match (exact match gets bonus)
    - Reliability (on-time percentage)
    
    Provide a clear ranking with justification.
    """
)

assignment_decision_agent = create_agent_node(
    agent_name="Assignment Decision Maker",
    system_prompt="""
    You are the Assignment Decision Maker. Your role is to:
    1. Review all scored drivers and their compatibility scores
    2. Make the final assignment decision
    3. Consider not just the highest score, but also:
       - Safety and compliance (HOS violations)
       - Operational efficiency
       - Customer satisfaction
    4. Provide clear reasoning for your decision
    5. Format the final assignment with all relevant details
    
    You have the authority to make the final decision. Be confident but explain your reasoning.
    Output the assignment in a clear, structured format.
    """
)

print("✓ Agentic nodes created:")
print("  - Load Analyzer")
print("  - Driver Search Specialist")
print("  - Matching Specialist")
print("  - Assignment Decision Maker")

## 8. Tool Execution Node

Handle tool calls made by agents.

In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import ToolMessage

def tool_node(state: AgenticDriverAssignmentState) -> AgenticDriverAssignmentState:
    """Execute tools requested by agents."""
    messages = state.get("messages", [])
    last_message = messages[-1] if messages else None
    
    if not last_message or not hasattr(last_message, 'tool_calls'):
        return state
    
    tool_calls = last_message.tool_calls
    if not tool_calls:
        return state
    
    print(f"\n[Tool Execution] Executing {len(tool_calls)} tool call(s)...")
    
    tool_messages = []
    for tool_call in tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tool_id = tool_call['id']
        
        # Find and execute the tool
        tool_func = next((t for t in tools if t.name == tool_name), None)
        if tool_func:
            print(f"  Executing: {tool_name}({tool_args})")
            result = tool_func.invoke(tool_args)
            print(f"  Result preview: {result[:200]}..." if len(result) > 200 else f"  Result: {result}")
            
            tool_messages.append(
                ToolMessage(
                    content=result,
                    tool_call_id=tool_id,
                    name=tool_name
                )
            )
    
    return {
        **state,
        "messages": tool_messages
    }

print("✓ Tool node created")

## 9. Build the LangGraph Workflow

In [ ]:
def should_continue(state: AgenticDriverAssignmentState) -> Literal["tools", "continue"]:
    """Determine if we should execute tools or continue to next agent."""
    messages = state.get("messages", [])
    last_message = messages[-1] if messages else None
    
    if last_message and hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return "continue"

# Create the workflow
workflow = StateGraph(AgenticDriverAssignmentState)

# Add agent nodes
workflow.add_node("load_analyzer", load_analyzer_agent)
workflow.add_node("driver_search", driver_search_agent)
workflow.add_node("matching_specialist", matching_specialist_agent)
workflow.add_node("assignment_decision", assignment_decision_agent)
workflow.add_node("tools", tool_node)

# Set entry point
workflow.set_entry_point("load_analyzer")

# Add conditional edges for tool calling
workflow.add_conditional_edges(
    "load_analyzer",
    should_continue,
    {
        "tools": "tools",
        "continue": "driver_search"
    }
)

workflow.add_edge("tools", "load_analyzer")  # After tools, return to agent

workflow.add_conditional_edges(
    "driver_search",
    should_continue,
    {
        "tools": "tools",
        "continue": "matching_specialist"
    }
)

workflow.add_conditional_edges(
    "matching_specialist",
    should_continue,
    {
        "tools": "tools",
        "continue": "assignment_decision"
    }
)

workflow.add_conditional_edges(
    "assignment_decision",
    should_continue,
    {
        "tools": "tools",
        "continue": END
    }
)

# Compile
app = workflow.compile()

print("✓ Agentic workflow compiled successfully!")

## 10. Visualize the Workflow

In [ ]:
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not visualize graph: {e}")
    print("\nWorkflow structure:")
    print("""    
    START
      ↓
    Load Analyzer ←→ Tools
      ↓
    Driver Search ←→ Tools
      ↓
    Matching Specialist ←→ Tools
      ↓
    Assignment Decision ←→ Tools
      ↓
    END
    """)

## 11. Execute the Agentic Workflow

### Test Case 1: Load L1001 (DryVan to Bangalore)

In [ ]:
print("\n" + "="*80)
print("🚀 EXECUTING AGENTIC WORKFLOW: Load L1001")
print("="*80)

# Initial state with task description
initial_state = {
    "load_id": "L1001",
    "messages": [
        HumanMessage(content="""
        Please assign the best driver for load L1001.
        
        Requirements:
        - Use semantic search to find suitable drivers
        - Calculate compatibility scores
        - Make an informed decision based on all factors
        - Provide clear reasoning for the assignment
        """)
    ],
    "load_details": None,
    "candidate_drivers": None,
    "scored_drivers": None,
    "final_assignment": None,
    "reasoning": [],
    "next_agent": None
}

# Execute workflow
try:
    result = app.invoke(initial_state)
    
    print("\n" + "="*80)
    print("✅ WORKFLOW COMPLETED")
    print("="*80)
    
    # Display reasoning chain
    print("\n📋 AGENT REASONING CHAIN:")
    print("="*80)
    for i, reasoning in enumerate(result['reasoning'], 1):
        print(f"\n{i}. {reasoning}")
    
except Exception as e:
    print(f"\n❌ Error executing workflow: {e}")
    import traceback
    traceback.print_exc()

### Test Case 2: Load L1002 (Reefer to Chennai)

In [ ]:
print("\n" + "="*80)
print("🚀 EXECUTING AGENTIC WORKFLOW: Load L1002")
print("="*80)

initial_state_2 = {
    "load_id": "L1002",
    "messages": [
        HumanMessage(content="""
        Please assign the best driver for load L1002.
        
        Requirements:
        - Use semantic search to find suitable drivers
        - Calculate compatibility scores
        - Make an informed decision based on all factors
        - Provide clear reasoning for the assignment
        """)
    ],
    "load_details": None,
    "candidate_drivers": None,
    "scored_drivers": None,
    "final_assignment": None,
    "reasoning": [],
    "next_agent": None
}

try:
    result_2 = app.invoke(initial_state_2)
    
    print("\n" + "="*80)
    print("✅ WORKFLOW COMPLETED")
    print("="*80)
    
    # Display reasoning chain
    print("\n📋 AGENT REASONING CHAIN:")
    print("="*80)
    for i, reasoning in enumerate(result_2['reasoning'], 1):
        print(f"\n{i}. {reasoning}")
    
except Exception as e:
    print(f"\n❌ Error executing workflow: {e}")
    import traceback
    traceback.print_exc()

## 12. Advanced: Test Semantic Search Capabilities

In [ ]:
print("\n🔍 Testing FAISS Semantic Search Capabilities")
print("="*80)

# Test different semantic queries
test_queries = [
    "Find drivers near Coimbatore with good reliability",
    "Experienced reefer drivers with high on-time performance",
    "Drivers with maximum hours of service available",
    "Closest available flatbed drivers"
]

for query in test_queries:
    print(f"\nQuery: '{query}'")
    print("-" * 80)
    
    results = driver_vectorstore.similarity_search_with_score(query, k=3)
    
    for i, (doc, score) in enumerate(results, 1):
        metadata = doc.metadata
        print(f"{i}. {metadata['name']} ({metadata['driver_id']}) - Score: {score:.4f}")
        print(f"   {metadata['city']} | {metadata['skill']} | HOS: {metadata['hos_remaining']}h | On-time: {metadata['on_time']}%")

## 13. Results Summary and Comparison

In [ ]:
display(Markdown("""
## 🎯 Summary: 100% Agentic System Benefits

### Key Features Implemented:

1. **Fully Autonomous Agents**
   - Each agent uses GPT-4 for reasoning
   - Agents can call tools independently
   - Natural language understanding and generation

2. **FAISS Vector Database**
   - Semantic search for drivers
   - Embedding-based similarity matching
   - Efficient retrieval at scale

3. **Agent-Tool Integration**
   - `search_available_drivers` - Semantic driver search
   - `get_load_details` - Load information retrieval
   - `calculate_compatibility_score` - Score calculation
   - `get_driver_by_id` - Driver detail lookup

4. **Multi-Agent Collaboration**
   - Load Analyzer → Driver Search → Matching → Assignment
   - Each agent specializes in one aspect
   - Agents communicate via shared state

5. **Explainable AI**
   - Every decision is explained
   - Reasoning chain is transparent
   - Score breakdowns provided

### Advantages Over Rule-Based Systems:

| Aspect | Rule-Based | Agentic (This System) |
|--------|------------|----------------------|
| Flexibility | Fixed rules | Adaptive reasoning |
| Search | Exact match | Semantic similarity |
| Decisions | Algorithmic | LLM-powered |
| Explanations | Limited | Natural language |
| Edge Cases | Hardcoded | Intelligent handling |
| Scalability | Vector DB enables millions of drivers | Limited |

### Expected Results:

Based on the scoring logic:

- **Load L1001 (DryVan)**: Driver D004 (Karthik R) - Score: 99.6
  - Distance: 10km (✓ bonus)
  - HOS: 8 hrs (✓ bonus)
  - Skill: DryVan (✓ match)
  - On-time: 96% (9.6 points)

- **Load L1002 (Reefer)**: Driver D001 (Arjun Kumar) - Score: 99.2
  - Distance: 12km (✓ bonus)
  - HOS: 6 hrs (✓ bonus)
  - Skill: Reefer (✓ match)
  - On-time: 92% (9.2 points)

### Next Steps:

1. **Production Integration**
   - Connect to real databases (PostgreSQL, MongoDB)
   - Add authentication and authorization
   - Implement caching and rate limiting

2. **Enhanced Features**
   - Real-time driver location tracking
   - Weather and traffic integration
   - Historical performance analysis
   - Predictive analytics for ETA

3. **Human-in-the-Loop**
   - Add approval workflows
   - Feedback mechanism for agent learning
   - Override capabilities for dispatchers

4. **Monitoring & Analytics**
   - Track assignment success rates
   - Monitor agent performance
   - A/B testing of different strategies
"""))

## 14. Interactive: Try Your Own Query

In [ ]:
def run_custom_assignment(load_id: str, custom_requirements: str = ""):
    """
    Run the agentic workflow with custom requirements.
    
    Args:
        load_id: The load to assign (L1001 or L1002)
        custom_requirements: Additional requirements or constraints
    """
    print(f"\n🚀 Custom Assignment Request for {load_id}")
    print("="*80)
    
    initial_state = {
        "load_id": load_id,
        "messages": [
            HumanMessage(content=f"""
            Please assign the best driver for load {load_id}.
            
            {custom_requirements if custom_requirements else 'Use standard criteria for assignment.'}
            
            Requirements:
            - Use semantic search to find suitable drivers
            - Calculate compatibility scores
            - Make an informed decision
            - Provide clear reasoning
            """)
        ],
        "load_details": None,
        "candidate_drivers": None,
        "scored_drivers": None,
        "final_assignment": None,
        "reasoning": [],
        "next_agent": None
    }
    
    try:
        result = app.invoke(initial_state)
        print("\n✅ Assignment completed!")
        return result
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

# Example: Run with custom requirements
# result = run_custom_assignment(
#     load_id="L1001",
#     custom_requirements="Prioritize drivers with the highest reliability, even if distance is greater."
# )

print("\n💡 Use the run_custom_assignment() function to test different scenarios!")
print("   Example: run_custom_assignment('L1001', 'Prioritize reliability over distance')")